# Day 22

link: https://adventofcode.com/2022/day/22

เก็บ input มาก่อน ข้อนี้ input แบ่งเป็น 2 ส่วน คือส่วน map กับ instruction เราแอบเติม comma ให้ instruction จะได้ split ง่ายๆ
ส่วน map นั้นก็เก็บมาตรงๆ เป็น string array 

In [1]:
input = IO.foreach('data/22.txt').to_a.map(&:chomp)
inst = input[-1].gsub('R', ',R,').gsub('L', ',L,').split(',')
map_s = input[0..-3]

nil

## Part 1

ใน part แรกนี้ โจทย์บอกว่า map จะ wrap around ก็คือช่องซ้ายสุดกับขวาสุดใน row เดียวกันจะเชื่อมกัน เช่นเดียวกับช่องบนสุดล่างสุดของแต่ละ column
เราก็เตรียมการโดยหาช่องซ้ายสุด/ขวาสุดของแต่ละ row เก็บลง lookup array `leftmost` กับ `rightmost` 
แล้วก็หาช่องบนสุด/ล่างสุด เก็บลง `topmost` กับ `bottommost` 
ตรงนี้ต้องระวังนิดนึงว่า กำแพง ('#') นั้นนับเป็นส่วนหนึ่งของ map ส่วนช่องว่าง (' ') นั้นไม่นับ
ดังนั้นในส่วนการหาตำแหน่งเพื่อ wraparound นี้เราต้อง search ใน string เพื่อหา character '.' หรือ '#'

จากนั้นเราสร้าง array ชื่อ `displacements` เพื่อเก็บการกระจัดในแต่ละทิศทาง เราใช้ coordinate ที่มีแกน r (บนลงล่าง) กับแกน c (ซ้ายไปขวา)
ดังนั้นทิศขวาคือ (0, 1) ลงคือ (1, 0) ซ้ายคือ (0, -1) ขึ้นคือ (-1, 0)

แล้วก็เริ่ม simulate ได้ จุดเริ่มต้นคือ row 0 ส่วน column ต้องหาที่ว่าง ('.') ช่องแรก direction ให้เริ่มไปทางขวา ก็คือ 0

ส่วน simulate จริงๆ ก็ไม่มีอะไรมาก หันขวาก็คือบวก 1 ไปที่ direction หันซ้ายก็ลบ 1 เดินหน้าก็วนเดินไปเรื่อยๆ จนเจอกำแพง (ซึ่งเราสามารถ break จาก loop การเดินได้ทันที) ถ้าหลุดออกนอก map ก็ไปโผล่อีกฝั่งหนึ่ง ตรงนี้ไม่ยากอะไร

พอเดินครบตาม instruction แล้วก็คำนวณ password ตามที่โจทย์บอก ระวังว่าโจทย์ใช้ 1-based index เลยต้องบวก 1 ให้ r, c ก่อนเอาไปคำนวณ

In [2]:
h = map_s.size
w = map_s.map(&:length).max

leftmost = h.times.map{|i| map_s[i].index(/[\.#]/) }
rightmost = h.times.map{|i| map_s[i].length - 1 - map_s[i].reverse.index(/[\.#]/) }
topmost = w.times.map{|i|  
  j = 0
  j += 1 while map_s[j].length < i || map_s[j][i] == ' '
  j
}
bottommost = w.times.map{|i|  
  j = h-1
  j -= 1 while map_s[j].length < i || map_s[j][i] == ' '
  j
}
  
displacements = [[0, 1], [1, 0], [0, -1], [-1, 0]]
direction = 0

r = 0
c = map_s[0].index('.')

inst.each{|a|
  if a == 'R'
    direction = (direction + 1) % 4
  elsif a == 'L'
    direction = (direction + 3) % 4
  else
    steps = a.to_i
    dr,dc = displacements[direction]
    steps.times{ 
      nr = r + dr
      nc = c + dc
      nr = topmost[c] if nr > bottommost[c]
      nr = bottommost[c] if nr < topmost[c]
      nc = leftmost[r] if nc > rightmost[r]
      nc = rightmost[r] if nc < leftmost[r]
      if map_s[nr][nc] == '.'
        r = nr
        c = nc
      else
        break
      end
    }
  end
}

def pwd(r, c, direction) = 1000 * (r + 1) + 4 * (c + 1) + direction

puts pwd(r, c, direction)

56372


## Part 2

part หลังนี้จัดว่ายากเลยทีเดียวสำหรับเรา โจทย์บอกว่า แผนที่ที่ให้มานั้นมันเอามาพับประกอบเป็นลูกบาศก์ได้!
ก็คือรีรันโจทย์เดิมแต่เปลี่ยนจาก flat surface มาอยู่บน cube แทน ดังนั้นมันไม่ต้องมีการ wrap around
เพราะว่าหน้าทั้ง 6 ของลูกบาศก์มันเชื่อมต่อกัน ไม่ต้องกระโดด warp ไปอีกข้างแล้ว

หรือจริงๆ เราอาจจะมองว่ามันยังมี warp อยู่ก็ได้ แต่มันไม่ได้อยู่ใน row หรือ column เดียวกันแล้ว
และนอกจากจะเปลี่ยนตำแหน่งแล้วยังอาจจะมีการเปลี่ยนทิศทางด้วย ขึ้นอยู่กับว่าลูกบาศก์มันถูกคลี่ออกมาเป็นแผนที่ยังไง

ถ้าเราจะ track ว่าออกจากจุดนี้แล้วต้องไปโผล่จุดนั้นในทิศทางนู้น มันก็พอทำได้อยู่ (แบบ manual) 
แต่เราคิดว่ามันค่อนข้างลำบากที่จะ automate สุดท้ายก็เลยตัดสินใจว่า เราทำโมเดลของ cube surface ขึ้นมาจริงๆ เลยน่าจะ generic กว่า

เริ่มจากทำ canonical representation ของ cube unwrapping ขึ้นมา เราคิดว่าแบบนี้น่าจะใช้งานง่ายที่สุด

```
    +---+
    | 0 |
+---+---+---+
| 4 | 1 | 5 |
+---+---+---+
    | 2 |
    +---+
    | 3 |
    +---+
```
ที่ว่าใช้งานง่ายก็คือ เราจะกำหนดให้ current surface เป็น surface หมายเลข 1 ทีนี้ถ้าเราเดินหลุดกรอบของ surface นี้ออกไป
มันก็จะเข้าไปอยู่ใน surface เบอร์ 0, 2, 4, หรือ 5 โดยที่เราไม่ต้องกังวลเรื่อง direction ว่ามันจะต้องหมุนซ้ายขวายังไงมั้ย การเช็ค
collision ก็จะทำได้ง่าย

แต่ที่เราต้องทำก็คือ ในเมื่อเราอยากจะให้ current surface อยู่ที่เบอร์ 1 ตลอดเวลา ดังนั้นต้อง track ว่าเมื่อเดินออกจากเบอร์ 1 ไปแล้ว
จะต้องสลับสับเปลี่ยน surface อย่างไรเพื่อให้มันกลับเข้าสู่ canonical form เหมือนเดิม

trick นึงที่เราใช้เพื่อดูว่า orientation ของ surface จะต้องปรับเปลี่ยนยังไง คือถ้ามีจุดที่เป็นมุมร่วมกันของ 3 surfaces
เราสามารถ fix มุมนั้น แล้วหมุน surface ไป 90 องศารอบมุมนั้นได้ ตัวอย่าง:
```
ᗷᗅ  ==>   ᗷ 
ᗞ         ᗞᗆ
```

เราก็จะ track transformation ของ surface ได้ดังนี้
- หลุดจาก 1 ไป 0: แถบ 0-1-2-3 ยังอยู่เหมือนเดิม แต่เราหมุนไปเป็น 3-0-1-2 เบอร์ 4 กับ 5 ต้องหมุนจากที่เชื่อมกับ 1 ไปเชื่อมกับ 0 จะได้ 4 ที่หมุนไป 90 องศา (ตามเข็ม) และ 5 ที่หมุนไป 270 องศา
- หลุดจาก 1 ไป 2: คิดคล้ายๆ กัน คือแถบ 0-1-2-3 จะหมุนไปเป็น 1-2-3-0, เบอร์ 4 หมุนลงมาเชื่อมกับ 2 (270 องศา), เบอร์ 5 หมุนลงมาเชื่อมกับ 2 เช่นกัน ที่ 90 องศา
- หลุดจาก 1 ไป 4: 4 จะมาแทนตำแหน่ง 1, 1 shift ไปทางขวา แทนตำแหน่ง 5, 0 ยังอยู่ที่เดิม แต่ต้องหมุนมาเชื่อมกับ 4 (270 องศา), 2 ยังอยู่ที่เดิม หมุนมาเชื่อมกับ 4 (90 องศา), ถ้าเรามองว่า 3 หมุนตาม 2 ไปด้วย ก็จะไปอยู่ตำแหน่งซ้ายล่างของ 4 (และหมุนไป 90 องศา) เราจับหมุนอีกทีขึ้นไปเชื่อมกับ 4 ก็จะได้ว่า 3 ไปแทนตำแหน่ง 4 (180 องศา), คิดในทางกลับกัน 5 ก็สามารถหมุนครั้งมาอยู่ที่ตำแหน่ง 3 (180 องศา)
- หลุดจาก 1 ไป 5: เหมือนข้างบนแต่กลับด้านกัน ดังนั้น 5 ไปแทน 1, 1 ไปแทน 4, 0 หมุนไป 90 องศา, 2 หมุนไป 270 องศา, 3 ไปแทนตำแหน่ง 5 (180 องศา), และ 4 ไปแทน 3 (180 องศา)

จะเห็นว่าเราต้องการ structure ที่สามารถรองรับการหมุนไปหมุนมานี้ได้ โดยที่เราสามารถ refer ถึงตำแหน่งบนหน้า cube เพื่อ track movement
ในขณะเดียวกันก็สามารถ refer ถึงตำแหน่งแท้จริงบน original 2-d map ได้ด้วย เพื่อที่จะเอามาคิด password ตอนจบ

เราสร้างคลาส `CubeSide` ขึ้นมาเพื่อทำหน้าที่นี้ โดยใส่ minimap ขนาด 50x50 เข้าไป แล้วตั้ง orientation เริ่มให้แกน x ชี้ลงล่าง
เราใช้ complex unit เป็น orientation ทำให้การหมุนไปมา เราก็แค่คูณด้วย i, -1, หรือ -i เท่านั้น
เรายังสามารถแปลงพิกัดไปมาระหว่างพิกัดบนหน้า cube surface กับพิกัดจริงบน map ได้ด้วยวิธีการทำนองเดียวกัน

จากนั้นก็สร้างคลาส `Cube` ซึ่งประกอบด้วย `CubeSide` หกชิ้น ตาม canonical form และมี method สำหรับหมุนไปมา ตามข้างบน

In [3]:
class CubeSide

  attr_accessor  :rows, :orientation

  def initialize(rows, offset_r, offset_c)
    @rows = rows
    @offset_r = offset_r
    @offset_c = offset_c
    @orientation = 1+0i

    @l = @rows.size
  end

  def rotate90!
    @orientation *= 1i
  end

  def rotate180!
    @orientation *= -1
  end

  def rotate270!
    @orientation *= -1i
  end

  def translate(r, c)
    z = Complex(r + 1, c + 1) * @orientation
    nr = z.real < 0 ? @l + z.real : z.real - 1
    nc = z.imaginary < 0 ? @l + z.imaginary : z.imaginary - 1
    [nr, nc]
  end

  def at(r, c)
    z = translate(r, c)
    @rows[z[0]][z[1]]
  end

  def absolute_position(r, c)
    z = translate(r, c)
    [@offset_r + z[0], @offset_c + z[1]]
  end

end

class Cube

  attr_accessor  :sides

  def initialize
    @sides = Array.new(6)
  end

  def turn_up!
    @sides = [3, 0, 1, 2, 4, 5].map{|i| @sides[i]}
    @sides[4].rotate90!
    @sides[5].rotate270!
  end

  def turn_down!
    @sides = [1, 2, 3, 0, 4, 5].map{|i| @sides[i]}
    @sides[4].rotate270!
    @sides[5].rotate90!
  end  

  def turn_left!
    @sides = [0, 4, 2, 5, 3, 1].map{|i| @sides[i]}
    @sides[0].rotate270!
    @sides[2].rotate90!
    @sides[3].rotate180!
    @sides[4].rotate180!
  end

  def turn_right!
    @sides = [0, 5, 2, 4, 1, 3].map{|i| @sides[i]}
    @sides[0].rotate90!
    @sides[2].rotate270!
    @sides[3].rotate180!
    @sides[5].rotate180!
  end

end
nil

ขั้นตอนการสร้าง `Cube` จริงๆ ขึ้นมานั้น ยังเป็นแบบ manual อยู่ คือเรานั่งดู map แล้วก็ hard code ไปเลยว่า
surface ไหน ต้องเอามาจากตำแหน่งไหนใน map เต็ม และต้องตั้ง orientation เป็นแบบไหน ตรงนี้เป็นการบ้าน ต้องไปหาวิธี automate เอาทีหลัง

ส่วนการ track movement ตามคำสั่งของโจทย์ ตรงนี้ไม่ได้ต่างจาก part 1 เท่าไหร่ จริงๆ ง่ายกว่าด้วยเพราะพิกัดเรามี fixed limit ว่าต้องอยู่ระหว่าง 0 กับ 50 ทั้งสองแกนเสมอ ที่ต้องเพิ่มมาก็คือต้องคอยหมุน cube เมื่อหลุดจาก surface ปัจจุบัน

ท้ายสุดเมื่อทำครบ instruction เราก็แปลง coordinate และ orientation สุดท้าย ไปเป็นพิกัดและทิศทางตาม map เริ่มต้น

In [4]:
block_size = 50

cube = Cube.new

def build_cube_side(map_s, r, c, block_size)
  br = r * block_size
  bc = c * block_size
  raw = block_size.times.map{|i|
    map_s[br + i][bc ... bc + block_size]
  }
  CubeSide.new(raw, br, bc)
end

# TODO: automate this!
cube.sides[1] = build_cube_side(map_s, 0, 1, block_size)
cube.sides[2] = build_cube_side(map_s, 1, 1, block_size)
cube.sides[3] = build_cube_side(map_s, 2, 1, block_size)
cube.sides[5] = build_cube_side(map_s, 0, 2, block_size)
cube.sides[0] = build_cube_side(map_s, 3, 0, block_size)
cube.sides[0].rotate270!
cube.sides[4] = build_cube_side(map_s, 2, 0, block_size)
cube.sides[4].rotate180!

r = 0
c = 0
direction = 0

inst.each{|a|
  if a == 'R'
    direction = (direction + 1) % 4
  elsif a == 'L'
    direction = (direction + 3) % 4
  else
    steps = a.to_i
    dr,dc = displacements[direction]
    steps.times{
      cs = 1
      nr = r + dr
      nc = c + dc
      begin nr = 0;              cs = 2 end if nr >= block_size
      begin nr = block_size - 1; cs = 0 end if nr < 0
      begin nc = 0;              cs = 5 end if nc >= block_size
      begin nc = block_size - 1; cs = 4 end if nc < 0
      
      if cube.sides[cs].at(nr, nc) == '.'
        r = nr
        c = nc
        case cs
        when 0 then cube.turn_up!
        when 2 then cube.turn_down!
        when 4 then cube.turn_left!
        when 5 then cube.turn_right!
        end
      else
        break
      end
    }
  end
}

ar, ac = cube.sides[1].absolute_position(r, c)
zf = Complex(*displacements[direction]) * cube.sides[1].orientation
af = displacements.index([zf.real, zf.imaginary])

puts pwd(ar, ac, af)

197047
